In [ ]:
%load_ext rpy2.ipython

from rpy2.robjects.packages import importr
import numpy as np
import pandas as pd
from chromax import Simulator
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import os
os.environ["KERAS_BACKEND"] = "jax"
import matplotlib
from jax import device_get
from scipy import stats

from typing import Dict, Any, List
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Lambda
import tensorflow as tf


In [ ]:

def generate_genetic_map(n_markers, n_chromosomes):
  df = pd.DataFrame(generate_marker_effects(n_markers=n_markers), columns=['Yield'])
  df['cM'] = np.random.uniform(0, 200, len(df))
  df['CHR.PHYS'] = '1A'
  df = df.sort_values(by='cM')
  df = df[['CHR.PHYS', 'cM', 'Yield']]
  # save df as csv under filename
  return df

def simplify_geneticmap(lst, qtl):
    # Select 'qtl' random indexes
    indexes_positive = random.sample(range(len(lst)), qtl)
    
    # Select 'qtl' random indexes not already selected
    remaining_indexes = set(range(len(lst))) - set(indexes_positive)
    indexes_negative = random.sample(remaining_indexes, qtl)

    # Modify the list
    for i in range(len(lst)):
        if i in indexes_positive:
            lst[i] = random.uniform(0.5, 1.0)  # Assign random float between 0.5 and 1.0
        elif i in indexes_negative:
            lst[i] = random.uniform(-0.5, -1.0)  # Assign random float between -0.5 and -1.0
        else:
            lst[i] = 0  # Assign 0

    return lst

def generate_population(n_pop=100, n_markers=500):
    """
    Generate a numpy array of randoms of length 500 with randomized 0, 1, or 2 at each position.
    It will generate 100 individuals based on n_pop.

    Returns: numpy array of size (n_pop, n_markers)
    """
    shape=(n_pop, n_markers, 2)
    # Define the elements to choose from and their associated probabilities
    elements = [0, 1, 2]
    probabilities = [1/3, 1/3, 1/3]  # equal probabilities for 0, 1, and 2

    # Generate the population
    population = np.random.choice(elements, size=(n_pop, n_markers), p=probabilities)

    return np.random.choice([True, False], size=shape)


def generate_marker_effects(n_markers=500, mu=0, sigma=0.1):
    """
    Generate a numpy array of marker effects with a normal distribution.

    Parameters:
    n_markers (int): Number of markers.
    mu (float): Mean of the distribution.
    sigma (float): Standard deviation of the distribution.

    Returns:
    numpy array of marker effects
    """

    # Generate the marker effects
    marker_effects = np.random.normal(mu, sigma, n_markers)

    return marker_effects

def parse_markerEffects(genetic_map, nChr):
    # Get the length of the genetic map
    length = len(genetic_map)

    # Create a new array for storing the chromosome number for each marker
    chr = [0] * length

    # Calculate the number of markers per chromosome
    markers_per_chr = length // nChr

    # Iterate over the range of the genetic map length
    for i in range(length):
        # Calculate the chromosome number and store it in the chr array
        chr[i] = i // markers_per_chr + 1

    return chr


def reshape_pop(maizeHaplo):
    reshapeHaplo = maizeHaplo.reshape(int((maizeHaplo.shape[0])/2),2,maizeHaplo.shape[1])
    reshapeHaplo = reshapeHaplo.transpose((0,2,1))
    return reshapeHaplo

def return_genetic_map_df(markerEffects, nChr, geneticMap):
    chr = parse_markerEffects(markerEffects, nChr)
    chr = [int(x[0]) for x in chr]
    trait = markerEffects
    pos = geneticMap
    # Assuming chr, trait, pos are your arrays
    df = pd.DataFrame({'CHR.PHYS': chr, 'Yield': trait, 'cM': pos})
    return df


In [ ]:
%%R
x <- seq(0, 2*pi, length.out=50)

In [ ]:
%R -o x

In [ ]:
x

In [ ]:
# %R install.packages("AlphaSimR")

In [ ]:
%%R
library("AlphaSimR")

In [ ]:
%%R
nInd = 10
nChr = 2
segSites = 5

founderGenomes = runMacs(nInd = nInd,
                         nChr = nChr,
                         segSites = segSites,
                         species = "MAIZE")


In [ ]:
%%R
SP = SimParam$new(founderGenomes)
SP$addTraitA(segSites)
# SP$setVarE(h2=.02)
pop = newPop(founderGenomes, simParam=SP)
ans = fastRRBLUP(pop, simParam=SP, useQtl=TRUE, use='gv')
ans@gv[[1]]@addEff
markerEffects = slot(slot(ans, "gv")[[1]], "addEff")
maizeHaplo = pullSegSiteHaplo(pop)
maizeGeno = pullSegSiteGeno(pop)
#cm positions of each marker
genMap = SP$genMap
geneticMap = unlist(genMap)

In [ ]:
%R -o maizeHaplo
%R -o maizeGeno
%R -o markerEffects
%R -o geneticMap
%R -o nInd
%R -o nChr
%R -o segSites

In [ ]:
nMarkers = segSites * nChr

In [ ]:
from statistics import mean

def split_and_average(lst, set_size):
    # lst is from get_baseline
    #average list
    lst = [x.mean() for x in lst]
    # Split the list into sublists
    split_lst = [lst[i:i + set_size] for i in range(0, len(lst), set_size)]

    # Calculate the average of the x-th element in each sublist
    averages = []
    for x in range(set_size):
        x_elements = [sublist[x] for sublist in split_lst if len(sublist) > x]
        averages.append(mean(x_elements))

    return averages

def sample_two_offspring_per_cross(dataset):
    """
    Samples two offspring per cross from the dataset.

    Parameters:
    dataset (np.ndarray): The input dataset with shape (n_cross, n_offspring, markers, diploid).

    Returns:
    np.ndarray: A new array with sampled offspring, shape (n_cross, 2, markers, diploid).
    """
    n_cross = dataset.shape[0]
    n_offspring_per_cross = 2
    # Create an array to store the indices of the offspring to sample
    indices = np.random.choice(dataset.shape[1], size=(n_cross, n_offspring_per_cross), replace=False)
    # Use advanced indexing to select two offspring per cross
    sampled_offspring = dataset[np.arange(n_cross)[:, None], indices, :, :]
    return sampled_offspring.reshape(-1, sampled_offspring.shape[2], sampled_offspring.shape[3])




def collapse_first_two_axes(dataset):
    """
    Collapses the first two axes of the dataset.

    Parameters:
    dataset (np.ndarray): The input dataset with shape (n_cross, n_offspring, markers, diploid).

    Returns:
    np.ndarray: A new array with the first two axes collapsed, shape (n_cross * n_offspring, markers, diploid).
    """
    # Calculate the new shape
    new_shape = (dataset.shape[0] * dataset.shape[1],) + dataset.shape[2:]
    # Reshape the dataset
    collapsed_dataset = dataset.reshape(new_shape)
    return collapsed_dataset





In [ ]:
# # Example usage:
# number_individuals = 50  # Replace with your actual number of individuals
# number_markers = 100     # Replace with your actual number of markers
# ploidy = 2              # Replace with your actual ploidy level
# batch_size = 5
# batch_id=1 

# initial_population = create_fake_population(batch_size, number_individuals, number_markers, ploidy)
# genetic_map_df = create_fake_geneticmap(number_markers)
# marker_strength = np.array(genetic_map_df['Yield'])

# dummy_score = np.random.rand(batch_size, number_individuals)

# actor_model = create_actor_model(number_individuals, number_markers, ploidy)
# critic_model = create_critic_model(number_individuals, number_markers, ploidy, actor_model)

In [ ]:
# policy = actor_model([initial_population,dummy_score])
# value_estimate = critic_model([initial_population, dummy_score])

In [ ]:
# #split the population in half based on the policy metric
# top_policy_parents = select_parents(policy[batch_id,:].numpy(), total_parents=int(initial_population.shape[1] /2))[1].numpy()

# bottom_policy_parents = np.arange(initial_population.shape[1])
# bottom_policy_parents = np.setdiff1d(bottom_policy_parents, top_policy_parents)

# top_policy_parents = initial_population[batch_id][top_policy_parents]
# bottom_policy_parents = initial_population[batch_id][bottom_policy_parents]


In [ ]:
# true_scores = calculate_scores(initial_population[batch_id], marker_strength)

In [ ]:
# calculate_scores(panmixia(bottom_policy_parents, 500), marker_strength).mean()

In [ ]:
# new_pop = panmixia(top_policy_parents, 50)
# new_scores = calculate_scores(new_pop, marker_strength)
# print(new_scores.mean())
# new_parents = scores2parents(new_scores,25)

In [ ]:
# new_pop = panmixia(new_pop[new_parents], 50)
# new_scores = calculate_scores(new_pop, marker_strength)
# print(new_scores.mean())
# new_parents = scores2parents(new_scores,2)

In [ ]:
# print(
#     f'''
#     Batch Size : {batch_size}
      
#     Population Size : {number_individuals}
#     Marker Count : {number_markers}

#     Population Input Shape : {initial_population.shape}
#     Scores Input Shape : {dummy_score.shape}

#     Policy Output Shape : {policy.shape}
#     Values Output Shape : {value_estimate.shape}
  
#     '''
#       )

In [ ]:
# FUNCTIONS FO SIMULATING THE ENVIRONMENT AND ACTIONS


def ppo_step(agent, parent_index,parent_score, cycle, total_cycles):
    """
    Takes an action in the environment and returns the next state, reward, and a done flag.

    Inputs:
    - agent: An instance of the BreederAgent class.
    - parent_index (batch_size, selected_parent_count): The indices of selected parents for breeding. FROM POLICY
    - cycle: The current cycle number within the episode.
    - total_cycles: The total number of cycles in an episode.

    Outputs:
    - step_data: A list of dictionaries, each containing the new state, reward, and done flag for a batch.
    """
    step_data = []
    #parents_index is calculated by the actor
    for c, x in enumerate(parent_index):  # for each batch
        parents_subpop = agent.current_population[c][x]
        mass_pop = panmixia(parents_subpop, total_offspring=agent.population_size * 5)
        mass_score = calculate_scores(mass_pop, agent.marker_strength)
        prev_score = agent.current_score[c].mean()
        reward = mass_score.mean() - prev_score

        # Sample a subset of the new population to form the next state
        new_pop_indices = np.random.choice(mass_pop.shape[0], size=agent.population_size, replace=False)
        new_pop = mass_pop[new_pop_indices]
        new_score = calculate_scores(new_pop, agent.marker_strength)

        done = (cycle == total_cycles - 1)
        batch_data = {
            'new_population': new_pop,
            'new_score':new_score,  # Store the actual next state
            'reward': reward,
            'parent_scores': parent_score,
            'previous_score': prev_score,
            'current_score': new_score.mean(),
            'done': done
        }
        step_data.append(batch_data)

    return step_data

"""
In summary, the advantage you are calculating is effectively measuring the relative benefit 
of the breeding decisions made by the agent compared to the expected outcome of the current
policy. It is a crucial component in training the agent, as it guides the optimization 
process towards decisions that yield better-than-average improvements in the plant population.

The advantage of a breeding decision (action) is the difference between the reward from that 
decision and the critic's estimated value of the population before breeding. If the advantage
is positive, it means that the breeding decision led to a better-than-expected improvement
in the population's mean score.
"""

def calculate_advantages(agent, episode_data):
    # Calculate Advantages

    advantages = []
    agent.gamma = .95
    # Assuming agent.gamma is the discount factor
    gamma = agent.gamma

    for cycle, step in enumerate(reversed(episode_data)):
        # The reversed list index starts from the last to the first
        reversed_cycle = (len(episode_data) - 1) - cycle

        # Get the reward and new values
        reward = np.array([x['reward'] for x in episode_data[reversed_cycle]])
        new_score = np.array([x['new_score'] for x in episode_data[reversed_cycle]])
        new_pop = np.array([x['new_population'] for x in episode_data[reversed_cycle]])
        new_value = agent.critic([new_pop, new_score])
        if reversed_cycle > 0:
            # If not the first cycle, get the previous values
            old_score = np.array([x['new_score'] for x in episode_data[reversed_cycle - 1]])
            old_pop = np.array([x['new_population'] for x in episode_data[reversed_cycle - 1]])
        else:
            # For the first cycle, use the initial population and score
            old_score = agent.initial_score
            old_pop = agent.initial_population

        # Calculate the value of the old population
        old_value = agent.critic([old_pop, old_score])

        # Calculate advantage
        delta = reward + gamma * new_value - old_value
        advantages.append(delta)


    # meant to evaluate how good the decision to select certain individuals for breeding 
    # (action) was, compared to an average or baseline selection strategy (state value)

    # Convert list of advantages to a numpy array
    advantages = np.array(advantages)

    # Remember to reverse the advantages array so it corresponds to the original order
    advantages = np.flip(advantages, axis=0)

    return advantages

def get_r(values):

    # Create an array of indices, same length as your list
    indices = np.array(range(len(values)))

    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(indices, values)
    return slope

def add_batchdim(arr):
    return np.expand_dims(arr, axis=0)

def create_fake_geneticmap(number_markers):
    # 'chr' will always be '1A' for every marker
    chr_array = ['1A'] * number_markers
    
    # 'yield': Create a marker_strength array with 1 float between -0.5 and +0.5 randomly
    # yield_array = np.random.poisson(np.random.randint(1,10), size=number_markers)
    poisson_values = np.random.poisson(np.random.randint(1, 10), size=number_markers)

    # Scale the Poisson values to the range [0, 1]
    scaled_poisson_values = poisson_values / np.max(poisson_values)

    # Stretch and shift the values to the range [-1, 1]
    yield_array = (scaled_poisson_values * 2) - 1
    
    # 'cM': create an array for number_markers length evenly sampled between 0 and 100
    cM_array = np.linspace(0, 100, num=number_markers)
    
    # Create the DataFrame with the auto-generated data
    df = pd.DataFrame({'CHR.PHYS': chr_array, 'Yield': yield_array, 'cM': cM_array*.01})
    
    return df

import numpy as np
from random import choice, shuffle

def create_fake_population(total_pops, number_individuals, number_markers, ploidy):
    # List of generation methods
    generation_methods = [
        lambda: np.random.randint(2, size=(number_individuals, number_markers, ploidy)),
        lambda: np.random.rand(number_individuals, number_markers, ploidy),
        lambda: np.random.standard_normal((number_individuals, number_markers, ploidy)),
        lambda: np.random.choice([0, 1], size=(number_individuals, number_markers, ploidy))
    ]
    
    populations = []
    for _ in range(total_pops):
        # Randomly select a generation method and generate the population
        gen_method = choice(generation_methods)
        population = gen_method()
        populations.append(population)
        
    # Combine all populations into a single numpy array
    combined_population = np.array(populations)
    return combined_population




def calculate_scores(population, marker_strength):
    """
    Calculates the additive score by matrix multipling the population (n,m) with the marker strengths (m,)
    """
    # Perform a dot product between the dosages and marker strength arrays
    dosages = np.sum(population,axis=2)
    scores = np.dot(dosages, marker_strength)
    return scores

def panmixia(selected_parents, total_offspring = 2):
    """ Handles the random crossing for us ; heuristic!

    selected_parents ( n , m , d )
    
    """
    n, m, d = selected_parents.shape
    offspring_target = total_offspring
    offspring_list = []

    while len(offspring_list) < offspring_target:
        # Randomly pick two parents without replacement
        parents_indices = np.random.choice(n, size=2, replace=False)
        parent1, parent2 = selected_parents[parents_indices]

        # Simulate random recombination for each marker
        offspring = np.zeros((m, d), dtype=parent1.dtype)
        for i in range(m):
            # Randomly choose one allele from each parent for each marker
            for j in range(d):
                parent_allele = np.random.choice([parent1[i, j], parent2[i, j]])
                offspring[i, j] = parent_allele

        # Add the new offspring to the offspring list
        offspring_list.append(offspring)

    # Convert offspring list to a numpy array with shape (offspring_target, m, d)
    offspring_array = np.array(offspring_list)
    return offspring_array


def scores2parents(scores,K):
    """
    hint: use output from calculate_scores
    """
    # Specify the number of top values you want (K)
    K = 5

    # Get the indices that would sort the array
    sorted_indices = np.argsort(scores)

    # Take the last K indices of the sorted indices array
    top_k_indices = sorted_indices[-K:]

    # Since argsort returns indices in ascending order, reverse to get the top values
    top_k_indices = top_k_indices[::-1]

    print("Indices of top K values:", top_k_indices)
    print("Top K values:", scores[top_k_indices])

    return top_k_indices


def select_parents(policy, total_parents):
    """
        input: Policy from actor ( metric value for each individual )
        returns : index of parents to be included in random_crosses for next step of breeding program
    """
    values, indices = tf.math.top_k(policy, total_parents)
    return values,indices

def create_actor_model(num_individuals, num_markers, ploidy):
    # Define the input layers
    population_input = Input(shape=(num_individuals, num_markers, ploidy), name='population_input')
    scores_input = Input(shape=(num_individuals,), name='scores_input')

    # Flatten the population input to process it with Dense layers
    flat_population = Flatten()(population_input)

    # Combine the flattened population and scores inputs
    combined_input = Concatenate()([flat_population, scores_input])

    # Define the hidden layers
    hidden1 = Dense(128, activation='relu')(combined_input)
    hidden2 = Dense(64, activation='relu')(hidden1)

    # Output layer with one scalar value per individual
    policy_output = Dense(num_individuals, activation='sigmoid', name='policy_output')(hidden2)

    # Create the actor model
    actor_model = Model(inputs=[population_input, scores_input], outputs=policy_output)

    return actor_model

def create_critic_model(num_individuals, num_markers, ploidy, actor_model):
    # Define the input layers
    population_input = Input(shape=(num_individuals, num_markers, ploidy), name='population_input')
    scores_input = Input(shape=(num_individuals,), name='scores_input')

    # Call the actor model to get the policy output for the given state
    policy_output = actor_model([population_input, scores_input])

    # Use Lambda layer to apply tf.stop_gradient to prevent backpropagation
    policy_output_no_gradient = Lambda(lambda x: tf.stop_gradient(x))(policy_output)

    # Flatten the population input to process it with Dense layers
    flat_population = Flatten()(population_input)

    # Combine the flattened population, scores, and policy outputs without gradients
    combined_input = Concatenate()([flat_population, scores_input, Flatten()(policy_output_no_gradient)])

    # Define the hidden layers
    hidden1 = Dense(128, activation='relu')(combined_input)
    hidden2 = Dense(64, activation='relu')(hidden1)

    # Output layer with a single value estimate for the state
    value_output = Dense(1, activation='linear', name='value_output')(hidden2)

    # Create the critic model
    critic_model = Model(inputs=[population_input, scores_input], outputs=value_output)

    return critic_model




class BreederAgent:
    """
    A PPO agent that learns to select parents for breeding programs using Actor-Critic method.
    """
    
    def __init__(self, config: Dict[str, Any]):
        """
        Initializes the breeding program with a configuration dictionary.
        
        Parameters:
        config (dict): Configuration dictionary containing all necessary parameters.
        """

        # Initialize the basic attributes from the config
        self.compute_baselines = config.get('compute_baselines', False)
        self.population_size = config['population_size']
        self.marker_count = config['marker_count']
        self.initial_population = config['initial_population']
        self.genetic_map = config['genetic_map']
        self.marker_strength = np.array(self.genetic_map['Yield'])
        self.initial_score = np.array([calculate_scores(x,self.marker_strength) for x in self.initial_population])

        self.ploidy = config['ploidy']


        # Initialize the Actor and Critic models
        self.actor = create_actor_model(num_individuals=self.population_size, num_markers=self.marker_count, ploidy=self.ploidy)
        self.critic = create_critic_model(num_individuals=self.population_size, num_markers=self.marker_count, ploidy=self.ploidy, actor_model=self.actor)


        #truncation and random baselines
        # self.truncation_baseline = [calculate_scores(x, self.marker_strength) for x in self.initial_population]
        # truncation_baseline = self.breeding_simulator.select(self.initial_population, k = int(self.initial_population.shape[0]/2))
        # self.truncation_baseline = self.breeding_simulator.GEBV(collapse_first_two_axes(self.breeding_simulator.random_crosses(truncation_baseline, n_crosses = truncation_baseline.shape[0], n_offspring=50))).mean()[0]

        # self.random_baseline = self.breeding_simulator.GEBV(collapse_first_two_axes(self.breeding_simulator.random_crosses(self.initial_population, n_crosses = self.initial_population.shape[0], n_offspring=50))).mean()[0]


    def reset_environment(self):
        self.current_population = self.initial_population
        self.current_scores = self.initial_score

# Example usage:
number_individuals = 50  # Replace with your actual number of individuals
number_markers = 100     # Replace with your actual number of markers
ploidy = 2              # Replace with your actual ploidy level
batch_size = 3
initial_population = create_fake_population(batch_size, number_individuals, number_markers, ploidy)
genetic_map_df = create_fake_geneticmap(number_markers)
marker_strength = np.array(genetic_map_df['Yield'])
fraction_selection = 10

config = {
    'compute_baselines': False,
    'initial_population': initial_population,
    'genetic_map': genetic_map_df,
    'population_size': number_individuals,
    'marker_count': number_markers,
    'heritability': .99,
    'episodes': 10,
    'cycles': 2,
    'learning_rate': .001,
    'replicates': 1,
    'ploidy':2,
}



In [ ]:
#Truncation Selection program

agent = BreederAgent(config)

# policy = actor_model([initial_population,dummy_score])
# value_estimate = critic_model([initial_population, dummy_score])


print([x.mean() for x in agent.initial_score])
# example using the raw scores (no actor/critic)
rep_scores = []
for e in range(2):
    scores = []
    agent.current_population = agent.initial_population
    agent.current_score = agent.initial_score
    for c in range(3):#action
        if c == 0:
            scores.append(agent.current_score.mean())

        selected_parents = [select_parents(x, total_parents = int(agent.population_size/fraction_selection))[1].numpy() for x in agent.current_score]
        subset_parents = [s[p] for s,p in zip(agent.current_population,selected_parents)]


        #view new environment
        new_pops = [panmixia(x, total_offspring=100) for x in subset_parents]
        sample_index = np.random.choice(new_pops[0].shape[0], size=agent.population_size, replace=False)
        new_pops = [x[sample_index] for x in new_pops]
        scored_pops = [calculate_scores(x,agent.marker_strength) for x in new_pops]
        new_pops = np.array(new_pops)
        scored_pops = np.array(scored_pops)

        # Add assertions here
        assert np.array(agent.current_population).shape == np.array(new_pops).shape, "Dimensions of current_population and new_pops do not match"
        assert np.array(agent.current_score).shape == np.array(scored_pops).shape, "Dimensions of current_score and scored_pops do not match"

        agent.current_population = new_pops
        agent.current_score = scored_pops
        scores.append(scored_pops.mean())
    rep_scores.append(scores)
    
[plt.plot(x,label=f'batch {c}') for c,x in enumerate(rep_scores)]

In [ ]:
[plt.hist(x,alpha=.5) for x in agent.initial_score]
plt.title('individual scores')
plt.show()
plt.hist(agent.marker_strength)
plt.title('marker strengths')
plt.show()

In [ ]:
#PPO Training Loop

#Initialize the environment, actor, and critic models, and set up any necessary tracking variables for rewards, scores, and baselines.

number_individuals = 50  # Replace with your actual number of individuals
number_markers = 100     # Replace with your actual number of markers
ploidy = 2              # Replace with your actual ploidy level
batch_size = 3

initial_population = create_fake_population(batch_size, number_individuals, number_markers, ploidy)
genetic_map_df = create_fake_geneticmap(number_markers)
marker_strength = np.array(genetic_map_df['Yield'])
selection_fraction = 3 #TOADD EVERYWEHRE( top 1/3)
config = {
    'compute_baselines': False,
    'initial_population': initial_population,
    'genetic_map': genetic_map_df,
    'population_size': number_individuals,
    'marker_count': number_markers,
    'heritability': .99,
    'episodes': 10,
    'learning_rate': .001,
    'replicates': 1,
    'ploidy':2,
}

agent = BreederAgent(config)

#Policy Based Step
agent.current_population = agent.initial_population
agent.current_score = agent.initial_score

# policy = agent.actor([agent.current_population, agent.current_score])
# value = agent.critic([agent.current_population, agent.current_score], agent.actor)

#Collect Single Episode of 5 Cycles With Current Policy
agent.episodes = 1
agent.cycles=10

#EPISODE EXPERIENCE COLLECTION
episode_data = []
values = []
old_policy_log_probs = []
for c in range(agent.cycles):
    #a score for each individual in the current population
    policy_logits = agent.actor([agent.current_population, agent.current_score])
    policy_probs = tf.nn.softmax(policy_logits)
    log_probs = tf.math.log(policy_probs + 1e-8)
    old_policy_log_probs.append(log_probs)
    #the estimated value of the current state (population and scores)
    value = agent.critic([agent.current_population, agent.current_score], agent.actor)
    parent_data = [select_parents(x, selection_fraction) for x in policy_logits]
    parent_score = [x[0].numpy() for x in parent_data] # contains policys metric of selected parents
    parent_index = [x[1].numpy() for x in parent_data] # contains index of selected parents
    breeding_step = ppo_step(agent,parent_index,parent_score, c,agent.cycles)
    new_pops = np.array([x['new_population'] for x in breeding_step])
    new_scores = np.array([x['new_score'] for x in breeding_step])
    agent.current_population = new_pops
    agent.current_score = new_scores
    for i in breeding_step:
        i['old_policy_log_probs'] = np.array([tf.nn.softmax(x).numpy() for x in policy_logits.numpy()])    
    episode_data.append(breeding_step)
    values.append([x.numpy()[0] for x in value])


advantages = calculate_advantages(agent,episode_data)
old_policy_log_probs = tf.stack(old_policy_log_probs, axis=0)
old_policy_log_probs = tf.transpose(old_policy_log_probs, perm=[1, 0, 2])

# Define the optimizer for both actor and critic networks
optimizer_actor = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'])
optimizer_critic = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'])

# Define the number of optimization epochs and the clipping parameter
optimization_epochs = 10
epsilon_clip = 0.2

# Convert advantages to a tensor
advantages = tf.convert_to_tensor(advantages, dtype=tf.float32)

# Flatten the advantages to match the policy's output shape
advantages = tf.reshape(advantages, [-1])

In [ ]:
# Optimization loop
for epoch in range(1):
    with tf.GradientTape() as tape_actor, tf.GradientTape() as tape_critic:
        # Initialize lists to store the ratios and value losses for each cycle
        policy_loss_components = []
        value_loss_components = []
        
        # Loop over each cycle in the recorded episode
        for cycle in range(agent.cycles):
            # Use the state and score at the specific cycle
            cycle_data = episode_data[cycle]

            recorded_pop = np.array([x['new_population'] for x in cycle_data])
            recorded_score = np.array([x['new_score'] for x in cycle_data])
            recorded__log_probs = np.array([x['old_policy_log_probs'] for x in cycle_data])
            # Recompute policy and values for the current weights
            current_policy_logits = agent.actor([recorded_pop, recorded_score], training=True)
            current_values = agent.critic([recorded_pop, recorded_score], training=True)
            current_values = tf.reshape(current_values, [-1])

In [ ]:
# Simplified optimization loop for a single step
with tf.GradientTape() as tape_actor, tf.GradientTape() as tape_critic:
    # Get the recorded information for the single step
    recorded_pop = np.array([x['new_population'] for x in episode_data[0]])
    recorded_score = np.array([x['new_score'] for x in episode_data[0]])
    recorded_log_probs = np.array([x['old_policy_log_probs'] for x in episode_data[0]])

    # Recompute policy logits for the current weights
    current_policy_logits = agent.actor([recorded_pop, recorded_score], training=True)
    
    # Compute new log probabilities using the current policy
    new_log_probs = tf.math.log(tf.nn.softmax(current_policy_logits))
    
    # Calculate the ratio between new and old policy probabilities
    ratio = tf.exp(new_log_probs - recorded_log_probs)
    
    # Assume advantages have been calculated already and are stored in a variable named `advantages`
    # Compute policy loss as the negative mean of the product of ratios and advantages
    policy_loss = -tf.reduce_mean(ratio * advantages[0])
    
    # Compute value loss using the critic model
    current_values = agent.critic([recorded_pop, recorded_score], training=True)
    value_loss = tf.reduce_mean((current_values - .01) ** 2)  # `returns` need to be calculated as per your setup

    # Calculate gradients and perform backpropagation
    policy_gradients = tape_actor.gradient(policy_loss, agent.actor.trainable_variables)
    value_gradients = tape_critic.gradient(value_loss, agent.critic.trainable_variables)

    # Apply gradients to the actor and critic models
    # optimizer_actor.apply_gradients(zip(policy_gradients, agent.actor.trainable_variables))
    optimizer_critic.apply_gradients(zip(value_gradients, agent.critic.trainable_variables))
